In [1]:
import os
from datetime import datetime

import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv import find_dotenv
from dotenv import load_dotenv

In [2]:
def get_soup(url):
    try:
        HEADERS = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
        response = requests.get(url, headers=HEADERS)
        return BeautifulSoup(response.content, "html.parser")
    except Exception as e:
        print(f"Failed to get soup from url {url}. Error: {e}")
        raise

In [3]:
crop = "milho"
URL = f"https://www.agrolink.com.br/noticias/cultura/{crop}/lista"
soup = get_soup(URL)

In [4]:
news_divs = soup.find_all('div', class_='col-auto col-blocks')

In [13]:
news_list = []

for div in news_divs:
    news_item = {}
    
    # Extrair o título da notícia
    news_title_tag = div.find('a', class_='news-list-main-link-title')
    if news_title_tag:
        news_item['news_title'] = news_title_tag.h4.text.strip()
        news_item['link'] = news_title_tag['href']
    
    # Extrair a data
    date_tag = div.find_all('a')[1]
    if date_tag:
        news_item['date'] = date_tag.text.strip().split(' ')[-2]
    
    # Extrair a descrição da notícia
    news_sub_tag = div.find('a', class_='news-list-main-link-description')
    if news_sub_tag:
        news_item['news_sub'] = news_sub_tag.text.strip()
    
    if news_item:
        news_list.append(news_item)

In [14]:
news_list

[{'news_title': 'Mercado de milho movimentado: Confira',
  'link': '/noticias/cultura/milho/mercado-de-milho-movimentado--confira_493542.html',
  'date': '31/07/2024',
  'news_sub': 'No Paraná os exportadores reavaliam programas'},
 {'news_title': 'Milho cai na B3: Entenda os motivos',
  'link': '/noticias/cultura/milho/milho-cai-na-b3--entenda-os-motivos_493541.html',
  'date': '31/07/2024',
  'news_sub': 'Na Bolsa de Chicago, o milho fechou em baixa com clima e boas condições das lavouras'},
 {'news_title': 'Avanço na regulação torna Brasil promissor em bioinsumos',
  'link': '/noticias/cultura/milho/avanco-na-regulacao-torna-brasil-promissor-em-bioinsumos_493538.html',
  'date': '31/07/2024',
  'news_sub': '“O crescimento dos bioinsumos no Brasil enfrenta desafios”'},
 {'news_title': 'Produção global e clima nos EUA impactam preço do milho',
  'link': '/noticias/cultura/milho/producao-global-e-clima-nos-eua-impactam-preco-do-milho_493529.html',
  'date': '30/07/2024',
  'news_sub': 